In [1]:
########Import Modules#########
import requests
from math import ceil
from bs4 import BeautifulSoup

In [2]:
######Start Up Function#########
def inform_input():
    global f_type
    global f_city
    global f_state
    print ('-------------Program start--------------')
    print ('Hello! This is a web scraper program that automatically collects the contact information of firms within a region.\n')
    print ('Note: This program will use public data from yellowpages.com\n')

    correct_input = False

    while correct_input != True:

        f_type = input('What type of firms do you want to search? (example: asset management)\nFirm type: ')
        print ('\n')

        f_city = input("Which city do you want to search? (example: Los Angeles)\nHint: If you want to search an entire state, then type 'ALL.'\nCity: ")
        print ('\n')

        f_state = input('Which state do you want to search? (example: CA)\nState: ')
        print ('\n')

        print ('Searching...'+f_type+' in '+f_city+','+f_state+'.\n')
        correct_input = input('Is the key words correct? [Y/N]\nAnswer: ')
        print ('\n')

        if correct_input.upper() == 'Y':
            correct_input = True

In [3]:
def get_url(firm_type,firm_city,firm_state):
    
    print ('creating url......')
    
    firm_type = firm_type.lower()
    firm_city = firm_city.lower()
    firm_state = firm_state.upper()

    firm_type = firm_type.split()
    firm_city = firm_city.split()

    ##basic form of url'https://www.yellowpages.com/search?search_terms=asset%20management&geo_location_terms=los%20angeles%2C%20CA&page=1'

    url = 'https://www.yellowpages.com/search?search_terms='

    url += firm_type.pop(0)
    while len(firm_type)!=0:
        url += '%20'+firm_type.pop(0)

    url += '&geo_location_terms='

    url += firm_city.pop(0)
    while len(firm_city)!=0:
        url += '%20'+firm_city.pop(0)
    
    url += '%2C%20'+firm_state+'&page='
    
    print ('creating url......done!')
    return url


In [4]:
def get_page_number(soup):
    
    print ('finding total page number......')
    total_results = soup.find_all('div',{"class":"pagination"})

    total_results = total_results[0].contents[0].text


    total_results = total_results.split()

    total_results = total_results.pop()

    total_results = int(total_results.replace('results',''))
    
    print ('finding total page number......'+str(ceil(total_results/30)))

    return ceil(total_results/30)



In [5]:
def get_contact_inform(url):
    
    global firm_name
    global firm_adr
    global firm_web
    global firm_phone
    r = requests.get(url)

    soup = BeautifulSoup(r.content,'lxml')

    g_data = soup.find_all('div', {'class':'info'})
    ######################get name#################
    for item in g_data:
        try:
            firm_name.append(str(item.contents[0].find_all('a',{'class':'business-name'})[0].text))
        except:
            firm_name.append('N/A')

    ######################get address#################
    for item in g_data:
        try:
            address = ''
            address += str(item.contents[1].find_all('span',{'class':'street-address'})[0].text) + ','
            address += str(item.contents[1].find_all('span',{'itemprop':'addressLocality'})[0].text).replace('\xa0','')
            address += str(item.contents[1].find_all('span',{'itemprop':'addressRegion'})[0].text) + ','
            address += str(item.contents[1].find_all('span',{'itemprop':'postalCode'})[0].text)

            firm_adr.append(address)
        except:
            firm_adr.append('N/A')

    ####################get phone number###################
    for item in g_data:
        try:
            firm_phone.append(str(item.contents[1].find_all('div',{'itemprop':'telephone'})[0].text))
        except:
            firm_phone.append('N/A')

    ####################get website###################
    for item in g_data:
        try:
            firm_web.append(str(item.contents[2].find_all('a',{'class':'track-visit-website'})[0].get('href')))
        except:
            firm_web.append('N/A')

In [6]:
#################main function##################

f_type = ''
f_city = ''
f_state = ''

firm_name = []
firm_adr = []
firm_web = []
firm_phone = []

inform_input()

-------------Program start--------------
Hello! This is a web scraper program that automatically collects the contact information of firms within a region.

Note: This program will use public data from yellowpages.com

What type of firms do you want to search? (example: asset management)
Firm type: icecream


Which city do you want to search? (example: Los Angeles)
Hint: If you want to search an entire state, then type 'ALL.'
City: san diego


Which state do you want to search? (example: CA)
State: CA


Searching...icecream in san diego,CA.

Is the key words correct? [Y/N]
Answer: y




In [7]:
if f_city.lower() != 'all':
    url_base = get_url(f_type,f_city,f_state)
        
    r = requests.get(url_base+'3')

    soup = BeautifulSoup(r.content,'lxml')

    page_number = get_page_number(soup)
    
    for number in range(1,page_number+1):
        
        url = url_base + str(number)
        get_contact_inform(url)
        print ('collecting information from page ' + str(number) + ' out of ' + str(page_number) + '......')
        
    print ('All the information is collected successfully!')

creating url......
creating url......done!
finding total page number......
finding total page number......12
collecting information from page 1 out of 12......
collecting information from page 2 out of 12......
collecting information from page 3 out of 12......
collecting information from page 4 out of 12......
collecting information from page 5 out of 12......
collecting information from page 6 out of 12......
collecting information from page 7 out of 12......
collecting information from page 8 out of 12......
collecting information from page 9 out of 12......
collecting information from page 10 out of 12......
collecting information from page 11 out of 12......
collecting information from page 12 out of 12......
All the information is collected successfully!


In [9]:
firm_adr

['3004 Juniper St,San Diego,CA,92104',
 '7007 Friars Rd,San Diego,CA,92108',
 '3840 Valley Centre Dr Ste 604,San Diego,CA,92130',
 '1602 State St,San Diego,CA,92101',
 '2121 El Cajon Blvd,San Diego,CA,92104',
 '910 Grand Ave Ste 108,San Diego,CA,92109',
 '9168 Mira Mesa Blvd,San Diego,CA,92126',
 '555 G St,San Diego,CA,92101',
 '2145 Fern St,San Diego,CA,92104',
 '2260 Callagan Hwy,San Diego,CA,92136',
 '900 4th Ave,San Diego,CA,92101',
 '711 4th Ave,San Diego,CA,92101',
 '600 W Broadway,San Diego,CA,92101',
 '859 W Harbor Dr,San Diego,CA,92101',
 '789 W Harbor Dr Ste 117,San Diego,CA,92101',
 'Seaport Village,San Diego,CA,92101',
 'Mission Hills Commons,San Diego,CA,92103',
 '3077 University Ave,San Diego,CA,92104',
 '4430 Maple St,San Diego,CA,92105',
 '3330 Adams Ave,San Diego,CA,92116',
 '4875 Manomet St,San Diego,CA,92113',
 '3740 Sports Arena Blvd,San Diego,CA,92110',
 '4073 Adams Ave,San Diego,CA,92116',
 '5131 College Ave,San Diego,CA,92115',
 '5157 College Ave,San Diego,CA,921